In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed June 22 22:40:00 2022

@author: Bradley

Backtesting framework
"""

In [1]:
from utils_new import *
from helper import *
from stats import *
from product_info import *

import warnings
warnings.filterwarnings('ignore')  

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%config InlineBackend.figure_format = 'svg'

%matplotlib inline 

In [2]:
# global settings
HEAD_PATH = "/home/yby/leadlag"
DATA_PATH = HEAD_PATH
SAVE_PATH = HEAD_PATH + "/result"
SIGNAL_PATH = HEAD_PATH
CORE_NUM = 48

In [3]:
# define the date list
import tushare as ts
begin = '2022-01-04'
end = '2022-06-16'
ts.set_token('0d86b9a1aa5a9ba78b7e3a2feba63242abaeae62c777f5742f6cf698')
pro = ts.pro_api()
df = ts.get_k_data('sh', start=begin, end=end)
date_list = df['date'].str.replace('-', '').to_list()
print(f"Total number of trading days used: {date_list.__len__()}")
date_list[:5]

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
Total number of trading days used: 107


['20220104', '20220105', '20220106', '20220107', '20220110']

In [4]:
# define the code list to be scanned
code_list1 = ['000776']
hushen300 = pd.read_csv("./data/hushen300.csv")
code_list2 = hushen300['代码'].apply(lambda x: "%06d"%x).tolist()
name_list = hushen300['名称'].tolist()
name_dict = OrderedDict(zip(code_list2, name_list))
code_dict = OrderedDict(zip(name_list, code_list2))
code_list2[:5]

['000001', '000002', '000063', '000066', '000069']

In [5]:
# define the grid used in the LLR
nums = [1, 2, 4, 8, 10, 15, 20, 30, 40, 50, 60, 120, 180, 240, 300]
whole_nums = list(map(lambda x: -x, nums))[::-1] + [0] + nums
whole_nums[:5]

[-300, -240, -180, -120, -60]

**Prepare the tick data for redis**

In [4]:
%%time
# only used when the redis is empty
# all_stock_data = prepare_all_data(code_list2, date_list)

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 11.2 µs


In [ ]:
# example: get one day data saved in redis
df_bytes_from_redis = SERVER.hget("tick_list", "20220429")
df_from_redis = pickle.loads(df_bytes_from_redis)
print(df_from_redis)

**Calculate lead-lag pairs using HY cross correlations**

In [5]:
%%time
hy_result = calc_all_pairs(code_list1, code_list2, date_list, nums)
hy_result

limit up/down occurs at 20220316 for stock 000776
Finish calculating HY correlation for 000776
CPU times: user 5min 4s, sys: 2min 21s, total: 7min 25s
Wall time: 11min 16s


OrderedDict([('000776',
              (array([[ 1.00000000e+00,  7.16502715e-04,  4.55323816e-04, ...,
                       -3.02808292e-03, -4.30848248e-04,  9.69603780e-05],
                      [ 1.00000000e+00,  2.91521682e-03,  1.33088630e-03, ...,
                        2.95592115e-03, -2.09585325e-03, -2.42234949e-03],
                      [ 1.00000000e+00,  4.19708110e-04,  2.51680449e-03, ...,
                        1.31530913e-03, -1.39736146e-04, -4.15025080e-05],
                      ...,
                      [ 1.00000000e+00,  5.73858715e-03,  2.38155172e-03, ...,
                        6.48503029e-04,  2.73299806e-03,  1.75707570e-03],
                      [ 1.00000000e+00,  4.33687866e-03,  5.13865882e-03, ...,
                        3.35145579e-03,  2.21614831e-04,  5.09484335e-03],
                      [ 1.00000000e+00,  5.43573000e-03,  2.96055838e-03, ...,
                        2.92186303e-03,  5.53831203e-03,  1.58610514e-03]]),
               array([[

In [6]:
with open(f"./temp_tick/{code_list1[0]}.pkl", "wb") as tf:
    pickle.dump(hy_result, tf) 

In [6]:
with open(f"./temp_tick/{code_list1[0]}.pkl", "rb") as tf:
    hy_result = pickle.load(tf)

In [7]:
result, result_std = hy_result['000776']
result.shape
result_std.shape

(31, 301)

(31, 301)

In [9]:
res_left, res_right = result-1.96*result_std/np.sqrt(date_list.__len__()), result+1.96*result_std/np.sqrt(date_list.__len__())

In [10]:
# In total how many stocks have limit up/down during this WHOLE period
result[np.isnan(result)].shape[0]/(2*nums.__len__()+1)

0.0

In [11]:
tag_name = name_dict[code_list1[0]]
temp = pd.DataFrame(result, columns=[tag_name] + name_list, index=whole_nums)
temp_left = pd.DataFrame(res_left, columns=[tag_name] + name_list, index=whole_nums)
temp_right = pd.DataFrame(res_right, columns=[tag_name] + name_list, index=whole_nums)
temp.drop([tag_name], axis=1, inplace=True)
temp_left.drop([tag_name], axis=1, inplace=True)
temp_right.drop([tag_name], axis=1, inplace=True)
temp.shape

(31, 299)

In [12]:
temp_left.head()

,平安银行,万 科Ａ,中兴通讯,中国长城,华侨城Ａ,TCL科技,中联重科,申万宏源,东方盛虹,美的集团,潍柴动力,徐工机械,云南白药,泸州老窖,古井贡酒,长安汽车,格力电器,长春高新,恒逸石化,中信特钢,京东方Ａ,中航西飞,长江证券,北新建材,一汽解放,五 粮 液,新 希 望,双汇发展,紫光股份,华东医药,浪潮信息,招商蛇口,新 和 成,华兰生物,大族激光,ST易购,分众传媒,苏 泊 尔,美年健康,紫光国微,三花智控,华峰化学,韵达股份,中环股份,宁波银行,正邦科技,中航光电,金风科技,科大讯飞,大华股份,歌尔股份,上海莱士,东方雨虹,洋河股份,海大集团,顺丰控股,北方华创,广联达,高德红外,海康威视,晶澳科技,赣锋锂业,天齐锂业,立讯精密,荣盛石化,三七互娱,百润股份,比亚迪,领益智造,龙佰集团,世纪华通,中公教育,完美世界,天赐材料,牧原股份,国信证券,坚朗五金,恩捷股份,凯莱英,视源股份,深南电路,鹏鼎控股,中国广核,乐普医疗,亿纬锂能,爱尔眼科,同花顺,东方财富,智飞生物,汇川技术,沃森生物,宋城演艺,阳光电源,晶盛机电,泰格医药,三环集团,芒果超媒,蓝思科技,先导智能,温氏股份,健帆生物,贝达药业,欧普康视,康泰生物,亿联网络,华大基因,英科医疗,宁德时代,康龙化成,迈瑞医疗,卓胜微,安克创新,稳健医疗,爱美客,金龙鱼,浦发银行,上海机场,包钢股份,华能国际,XD华夏银,民生银行,上港集团,宝钢股份,华能水电,中国石化,南方航空,中信证券,三一重工,招商银行,保利发展,中国联通,国投资本,人福医药,同仁堂,上汽集团,国金证券,北方稀土,中国东航,重庆啤酒,金发科技,中国船舶,天坛生物,中国巨石,生益科技,复星医药,恒瑞医药,万华化学,白云山,恒力石化,浙江龙盛,江西铜业,金地集团,国电南瑞,XD华鲁恒,片仔癀,通威股份,中金黄金,贵州茅台,山东黄金,恒生电子,长电科技,海螺水泥,用友网络,青岛啤酒,绿地控股,豫园股份,福耀玻璃,海尔智家,华域汽车,闻泰科技,中航沈飞,通策医疗,国电电力,山西汾酒,海通证券,宝信软件,上海临港,中炬高新,国投电力,伊利股份,航发动力,长江电力,三峡能源,中泰证券,江苏银行,杭州银行,东方证券,宝丰能源,招商证券,大秦铁路,南京银行,隆基绿能,春秋航空,中信建投,中国神华,恒立液压,财通证券,中国国航,工业富联,新城控股,天风证券,兴业银行,北京银行,中国铁建,国泰君安,君正集团,陕西煤业,上海银行,环旭电子,红塔证券,广汽集团,农业银行,中国平安,中国人保,交通银行,新华保险,三六零,兴业证券,中国中铁,工商银行,中国铝业,中国太保,上海医药,中国中冶,中国人寿,长城汽车,邮储银行,中国建筑,中国电建,华泰证券,中银证券,中国卫通,XD中国电,中国中车,光大证券,星宇股份,中国交建,中海油服,京沪高铁,光大银行,成都银行,中国石油,福莱特,中国能建,正泰电器,浙商证券,中国银河,中国中免,中煤能源,紫金矿业,方正证券,浙商银行,中远海控,永辉超市,建设银行,玲珑轮胎,中国核电,中国银行,中国重工,南京证券,中金公司,中信银行,中科曙光,甘李药业,汇顶科技,公牛集团,大参林,药明康德,合盛硅业,海天味业,浙江鼎力,今世缘,万泰生物,科沃斯,韦尔股份,绝味食品,安图生物,璞泰来,DR华友钴,福斯特,欧派家居,金域医学,晨光股份,益丰药房,兆易创新,洛阳钼业,东鹏饮料,澜起科技,中国通号,中微公司,传音控股,金山办公,沪硅产业-U,石头科技,华熙生物,华润微,奇安信-U,天合光能,中芯国际
-300,-0.003957,-0.002979,-0.000623,-0.001057,-0.003910,-0.003149,-0.002094,-0.000222,0.000132,-0.003839,-0.002253,-0.001131,-0.002019,-0.000285,-0.002030,-0.002506,-0.001574,-0.003195,-0.000926,-0.000856,-0.002402,-0.002227,-0.001496,-0.001024,-0.003715,-0.001777,-0.002727,-0.003414,-0.002354,0.000956,-0.003312,-0.002626,-0.006560,-0.002028,-0.002169,-0.006254,-0.001617,-0.001709,-0.003928,-0.002521,-0.003529,-0.003237,-0.003414,-0.001503,-0.000739,-0.001908,-0.006713,-0.001027,-0.001956,0.000547,0.001671,-0.001371,-0.000492,-0.003060,-0.006604,-0.002857,-0.002893,0.000691,-0.003902,-0.000443,-0.001550,-0.001288,-0.001913,-0.004351,-0.001553,-0.001434,-0.000697,-0.002380,-0.003370,-0.003435,-0.003377,-0.001570,-0.005928,-0.003774,-0.003088,-0.001774,-0.005063,-0.003253,-0.001026,-0.003792,-0.003924,-0.002026,-0.003755,-0.002875,-0.002047,-0.003288,-0.002150,-0.001545,-0.003510,-0.003937,-0.001528,0.000567,-0.004201,-0.001928,-0.002027,-0.009228,-0.004149,-0.003467,-0.001040,-0.000912,-0.000709,-0.005026,-0.004008,-0.002001,-0.003455,-0.004569,-0.004302,-0.001205,-0.004207,-0.002490,-0.001603,-0.002468,-0.004742,-0.004057,-0.001623,-0.002412,-0.000249,-0.006632,-0.002393,-0.002716,-0.001883,-0.000558,-0.003632,-0.003141,-0.003111,-0.003356,0.000898,-0.004816,-0.003538,-0.003539,-0.000939,-0.005077,-0.006093,-0.001823,-0.000522,0.000314,-0.002291,-0.000774,-0.003382,-0.001016,-0.002653,-0.002661,-0.002262,-0.001484,-0.005618,0.001024,-0.005266,-0.003835,-0.003333,-0.005161,-0.002168,-0.003056,-0.003158,-0.004226,-0.005825,0.000151,-0.002457,-0.002754,-0.001719,0.001600,-0.004397,-0.002428,-0.002876,-0.000881,-0.003300,-0.002848,0.000906,-0.002830,-0.000074,-0.002219,-0.006613,-0.002708,-0.003170,-0.001302,0.000809,-0.002100,-0.000744,-0.004186,-0.001928,-0.000140,-0.005890,-0.001357,-0.002948,-0.003489,-0.002399,-0.003240,-0.003310,-0.005284,-0.000510,-0.000785,-0.002062,-0.006133,-0.002076,-0.001386,-0.000976,-0.003286,-0.003096,-0.002570,-0.004312,-0.000991,0.002198,-0.000526,-0.006045,-0.001006,0.001731,-0.002680,-0.004467,-0.001374,-0.002355,0.000401,-0.004038,-0.002797,-0.000879,-0.001776,-0.001500,0.002187,-0.001580,-0.002987,-0.002378,-0.004589,-0.002943,0.001305,-0.003561,-0.001621,-0.002265,-0.006052,-0.001355,-0.003047,0.000051,0.000105,-0.001651,0.000753,-0.003446,-0.002739,-0.000451,-0.006571,-0.001084,-0.008628,-0.001847,-0.003705,-0.002066,-0.001831,-0.003799,-0.006153,0.002371,-0.001253,-0.003080,-0.000886,-0.004304,-0.004233,-0.002713,-0.006519,0.00229

In [26]:
llr = (temp.loc[1:]**2).sum()/(temp.loc[:0]**2).sum()
pd.DataFrame(llr, columns=['LLR'])

,LLR
平安银行,1.309618
万 科Ａ,0.677064
中兴通讯,1.036453
中国长城,1.431400
华侨城Ａ,0.991492
...,...
华熙生物,1.277395
华润微,1.270941
奇安信-U,1.556489
天合光能,0.771272


In [14]:
temp_lead = temp[[col for col in temp.columns if llr[col] < 1]]
temp_lag = temp[[col for col in temp.columns if llr[col] > 1]]

temp_lead_l = temp_left[[col for col in temp.columns if llr[col] < 1]].loc[:-1]
temp_lag_l = temp_left[[col for col in temp.columns if llr[col] > 1]].loc[1:]

temp_lead_r = temp_right[[col for col in temp.columns if llr[col] < 1]].loc[:-1]
temp_lag_r = temp_right[[col for col in temp.columns if llr[col] > 1]].loc[1:]


ori_cor_lead = temp_lead.loc[0]
ori_cor_lag = temp_lag.loc[0]

temp_lead = temp_lead.loc[:-1]
temp_lag = temp_lag.loc[1:]

trial_lead = temp_lead.describe().T
trial_lag = temp_lag.describe().T

trial_lead['ori_corr'] = ori_cor_lead
trial_lag['ori_corr'] = ori_cor_lag
trial_lead['llr'] = llr[llr < 1]
trial_lag['llr'] = llr[llr > 1]

trial_lag.shape
trial_lead.shape

(176, 10)

(123, 10)

In [15]:
trial_lead['rank'] = (trial_lead['ori_corr'].rank(ascending=False) + trial_lead['mean'].rank(ascending=False)).rank(ascending=True)
trial_lead.sort_values(['rank'], ascending=True).head(20)

,count,mean,std,min,25%,50%,75%,max,ori_corr,llr,rank
中信证券,15.0,0.021687,0.023751,-0.000424,0.002490,0.011729,0.032319,0.073723,0.075659,0.675903,1.0
东方证券,15.0,0.019391,0.021992,0.000417,0.001740,0.013987,0.028331,0.068215,0.072778,0.876725,2.5
中国平安,15.0,0.019813,0.021245,-0.002566,0.002971,0.013324,0.032370,0.067227,0.070314,0.804598,2.5
招商银行,15.0,0.019074,0.020065,-0.002091,0.002297,0.013490,0.028081,0.065161,0.065489,0.574957,4.5
华泰证券,15.0,0.019063,0.021786,-0.000712,0.002516,0.008418,0.028029,0.065503,0.067216,0.850737,4.5
国泰君安,15.0,0.017959,0.020426,-0.000826,0.002608,0.008424,0.028437,0.058518,0.061855,0.879989,6.0
五 粮 液,15.0,0.024972,0.030130,0.001512,0.003642,0.009589,0.035651,0.086356,0.050228,0.764735,7.0
中国太保,15.0,0.017111,0.017183,-0.001788,0.003779,0.010592,0.025844,0.050079,0.054691,0.735295,8.0
恒瑞医药,15.0,0.016305,0.016511,0.000010,0.004271,0.010766,0.024336,0.054151,0.057378,0.775793,9.0
隆基绿能,15.0,0.015932,0.017117,-0.001657,0.002706,0.010638,0.025544,0.057578,0.056940,0.715163,10.5


In [16]:
trial_lag['rank'] = (trial_lag['ori_corr'].rank(ascending=False) + trial_lag['mean'].rank(ascending=False)).rank(ascending=True)
trial_lag.sort_values(['rank'], ascending=True).head(20)

,count,mean,std,min,25%,50%,75%,max,ori_corr,llr,rank
平安银行,15.0,0.056403,0.059752,0.003002,0.009767,0.022509,0.102286,0.156618,0.132423,1.309618,1.0
TCL科技,15.0,0.044803,0.048438,0.003487,0.008604,0.016664,0.076664,0.129937,0.112725,1.111293,2.0
中兴通讯,15.0,0.042140,0.047189,0.002708,0.007006,0.014743,0.070098,0.127384,0.112672,1.036453,3.0
中国长城,15.0,0.039856,0.043974,0.001727,0.005832,0.014514,0.075561,0.112991,0.091965,1.431400,4.0
中金公司,15.0,0.030190,0.027229,0.003078,0.007813,0.017210,0.047931,0.081115,0.074637,1.199339,5.0
东方财富,15.0,0.034580,0.036543,0.004404,0.008200,0.014440,0.054366,0.106498,0.061796,1.095781,6.5
中信建投,15.0,0.025281,0.026709,-0.000666,0.004194,0.013276,0.042618,0.074038,0.070988,1.009064,6.5
浙商证券,15.0,0.022926,0.022815,0.000943,0.005260,0.013357,0.034707,0.067936,0.063159,1.161410,8.0
同花顺,15.0,0.034617,0.037065,0.002800,0.005277,0.016157,0.056720,0.102618,0.054417,1.553896,9.0
新华保险,15.0,0.021524,0.021480,-0.000045,0.004833,0.013492,0.035311,0.060795,0.058981,1.086239,10.0


In [17]:
t = temp_lead_l.T
key = -4
# t[t[key] > 0].sort_values([key], ascending=False).head(20)
t.sort_values([key], ascending=False).head(20)

,-300,-240,-180,-120,-60,-50,-40,-30,-20,-15,-10,-8,-4,-2,-1
五 粮 液,-0.001777,-0.002157,-0.001608,0.001785,-0.002937,0.004890,0.007284,0.002753,0.012769,0.004237,0.023285,0.036211,0.054594,0.077173,0.077173
比亚迪,-0.002380,-0.002547,0.000318,-0.002137,-0.003158,-0.001535,0.007212,0.001837,0.006085,0.003702,0.022884,0.033210,0.050172,0.064359,0.064359
泸州老窖,-0.000285,-0.000880,-0.000046,-0.004459,-0.005238,0.002435,0.002449,-0.000602,0.004777,0.006395,0.019507,0.031146,0.048998,0.068990,0.068990
赣锋锂业,-0.001288,-0.003109,-0.005188,-0.003518,0.001096,0.005827,0.009416,0.004415,0.011750,0.002154,0.018260,0.034172,0.048571,0.065873,0.065873
亿纬锂能,-0.002047,-0.004339,-0.003702,0.000071,-0.002044,0.005282,0.005786,-0.000097,0.010364,0.007218,0.018649,0.029495,0.043949,0.061474,0.061474
立讯精密,-0.004351,-0.006412,-0.002908,-0.003988,0.000714,0.001957,0.004778,0.000154,0.008966,0.002035,0.020447,0.028914,0.043398,0.058672,0.058672
中信证券,0.000898,-0.002973,-0.003779,-0.003973,-0.004029,0.001170,0.006796,0.007435,0.016439,0.019728,0.021828,0.032820,0.043278,0.066644,0.054290
天赐材料,-0.003774,-0.003263,0.000071,-0.002197,-0.002830,0.005052,0.007579,0.002334,0.007605,0.002384,0.016805,0.023276,0.042940,0.065592,0.065592
天齐锂业,-0.001913,-0.002416,-0.002822,-0.002923,0.001431,-0.000005,0.007372,-0.001420,0.008784,0.002641,0.017425,0.026348,0.042268,0.057002,0.057002
洋河股份,-0.003060,-0.000518,-0.003564,-0.002184,-0.002154,0.002303,0.005810,0.002313,0.004014,0.000975,0.016488,0.025721,0.040585,0.057670,0.057670


In [18]:
t = temp_lag_l.T
key = 4
# t[t[key] > 0].sort_values([key], ascending=False).head(20)
t.sort_values([key], ascending=False).head(20)

,1,2,4,8,10,15,20,30,40,50,60,120,180,240,300
平安银行,0.142990,0.145117,0.139777,0.103306,0.081985,0.042573,0.025542,0.015362,0.016235,0.015115,0.008424,-0.002206,-0.000482,-0.002098,-0.000987
TCL科技,0.116708,0.118815,0.111526,0.077334,0.058747,0.029495,0.015791,0.010464,0.010927,0.011251,0.006137,-0.001460,0.000882,0.000459,-0.000168
中兴通讯,0.116609,0.117586,0.105798,0.071088,0.053145,0.025666,0.016735,0.008441,0.009836,0.010169,0.003753,-0.001953,0.000853,-0.001344,-0.000821
中国长城,0.098230,0.100831,0.101885,0.075437,0.056946,0.028558,0.014450,0.007562,0.007135,0.009498,0.004163,-0.002989,-0.003456,-0.002635,-0.003479
同花顺,0.093915,0.093915,0.079715,0.058849,0.042102,0.010094,0.025763,0.006748,0.015058,0.011574,-0.000123,-0.000992,0.000930,-0.000510,0.001386
东方财富,0.097959,0.097959,0.069762,0.053846,0.042940,0.012911,0.025301,0.008122,0.009375,-0.001050,0.000735,0.003866,0.002918,0.003575,0.006051
中金公司,0.073573,0.063350,0.066155,0.045062,0.041271,0.028324,0.020873,0.012634,0.012282,0.007228,0.005331,-0.001404,0.001867,0.000818,0.001277
中信建投,0.066786,0.062169,0.058556,0.042718,0.031827,0.022324,0.012444,0.009356,0.007236,0.000748,0.000600,-0.004889,-0.000742,-0.004046,-0.001358
浙商证券,0.060897,0.051853,0.051983,0.028610,0.030346,0.022320,0.012667,0.009138,0.003538,0.003134,0.002852,-0.000393,-0.000324,-0.003033,0.000028
光大证券,0.054442,0.051310,0.051195,0.028115,0.025906,0.015448,0.011695,0.008878,0.003290,0.004136,0.005135,-0.000313,-0.000924,-0.003303,-0.004230


**Arbitrage backtesting**

In [8]:
targ_date = '20220617'

In [9]:
# product_x: leader 
# product_y: lagger (also the target asset)
product_x = code_dict['中信证券']
product_y = "000776"

data_x = get_tick(product_x, targ_date)
data_y = get_tick(product_y, targ_date)

In [10]:
data_x.head()

,date,code,server_time,local_time,preclose,open,high,low,last,upper_limit,lower_limit,volume,turnover,iopv,ask_price1,ask_volume1,ask_price2,ask_volume2,ask_price3,ask_volume3,ask_price4,ask_volume4,ask_price5,ask_volume5,ask_price6,ask_volume6,ask_price7,ask_volume7,ask_price8,ask_volume8,ask_price9,ask_volume9,ask_price10,ask_volume10,bid_price1,bid_volume1,bid_price2,bid_volume2,bid_price3,bid_volume3,bid_price4,bid_volume4,bid_price5,bid_volume5,bid_price6,bid_volume6,bid_price7,bid_volume7,bid_price8,bid_volume8,bid_price9,bid_volume9,bid_price10,bid_volume10,time,midquote,delta_quote
189,20220617,600030.SSE,2022-06-17 09:30:00,2022-06-17 09:30:00.000,20.82,20.6,20.60,20.60,20.60,22.9,18.74,1072805,22099783,0,20.61,8000,20.62,84600,20.63,23900,20.64,2500,20.65,61000,20.66,21860,20.67,1800,20.68,52900,20.69,4600,20.70,59260,20.60,66495,20.59,7200,20.58,24100,20.57,16600,20.56,13700,20.55,21600,20.54,5900,20.53,14500,20.52,56000,20.51,224300,09:30:00,3.025534,NaN
190,20220617,600030.SSE,2022-06-17 09:30:02,2022-06-17 09:30:02.000,20.82,20.6,20.62,20.52,20.52,22.9,18.74,1775100,36556269,0,20.60,42615,20.61,14500,20.62,38300,20.63,4800,20.64,2800,20.65,51900,20.66,21860,20.67,1600,20.68,52200,20.69,12000,20.52,94000,20.51,342300,20.50,226600,20.49,8000,20.48,43400,20.47,3200,20.46,12900,20.45,24100,20.44,8200,20.43,19600,09:30:02,3.023347,-0.002186
191,20220617,600030.SSE,2022-06-17 09:30:05,2022-06-17 09:30:05.000,20.82,20.6,20.65,20.52,20.60,22.9,18.74,2101300,43276737,0,20.65,21460,20.66,21760,20.67,1600,20.68,53400,20.69,17100,20.70,59860,20.71,15300,20.72,3400,20.73,3900,20.74,3100,20.60,42100,20.59,7500,20.58,15300,20.57,2200,20.56,26000,20.55,9900,20.53,24300,20.52,111400,20.51,362400,20.50,257900,09:30:05,3.026504,0.003156
192,20220617,600030.SSE,2022-06-17 09:30:09,2022-06-17 09:30:09.000,20.82,20.6,20.66,20.52,20.62,22.9,18.74,2528290,52083377,0,20.66,12120,20.67,1000,20.68,53300,20.69,32600,20.70,58100,20.71,15300,20.72,3400,20.73,71800,20.74,3100,20.75,8300,20.62,200,20.61,21395,20.60,385015,20.59,47900,20.58,25700,20.57,3100,20.56,53100,20.55,21100,20.54,300,20.53,24100,09:30:09,3.027231,0.000727
193,20220617,600030.SSE,2022-06-17 09:30:11,2022-06-17 09:30:11.000,20.82,20.6,20.70,20.52,20.69,22.9,18.74,2871165,59160108,0,20.70,51420,20.71,15200,20.72,3400,20.73,72900,20.74,3100,20.75,7400,20.76,46400,20.77,3100,20.78,10700,20.79,49575,20.69,100,20.63,2400,20.62,37650,20.61,4700,20.60,339985,20.59,52200,20.58,33800,20.57,15300,20.56,61000,20.55,27100,09:30:11,3.029892,0.002661


In [26]:
os.makedirs(HEAD_PATH+f"/comb_time/{product_x}_{product_y}", exist_ok=True)

In [33]:
# 使用计数器Counter
from collections import Counter
def get_shfe_comb_pos(date, product_x, product_y, HEAD_PATH):
    data_x = get_tick(product_x, date)
    data_y = get_tick(product_y, date)
    comb_time = np.intersect1d(data_x["server_time"], data_y["server_time"])
    comb_time_set = set(comb_time)
    time_x = [element in comb_time_set for element in data_x["server_time"]]
    time_y = [element in comb_time_set for element in data_y["server_time"]]
    if len(comb_time_set)!=len(time_x) or len(comb_time_set)!=len(time_y):
        count_x = Counter(data_x["server_time"])
        count_y = Counter(data_y["server_time"])
        good_x = [count_x[x]==1 and count_y[x]==1 for x in data_x["server_time"]]
        good_y = [count_x[x]==1 and count_y[x]==1 for x in data_y["server_time"]]
        time_x = (time_x and good_x)
        time_y = (time_y and good_y)
    save([time_x, time_y], HEAD_PATH+"/comb_time/"+product_x+"_"+product_y+"/"+date)

'555555'

In [ ]:
%%time
result = parLapply(CORE_NUM, all_dates, get_shfe_comb_pos, product_x=product_x, product_y=product_y, HEAD_PATH=HEAD_PATH)

In [ ]:
def par_get_arb_all_signal(signal_name, file_list, product_x, product_y, period, HEAD_PATH=HEAD_PATH, SAVE_PATH=SAVE_PATH):
    n_files = len(file_list)
    all_signal = np.array([])
    for file in file_list:
        S_x = load(SAVE_PATH+"/factor_pkl/"+product_x+"/"+signal_name+"/"+file)
        S_y = load(SAVE_PATH+"/factor_pkl/"+product_y+"/"+signal_name+"/"+file)
        [time_x, time_y] = load(HEAD_PATH+"/comb_time/"+product_x+"_"+product_y+"/"+file)
        signal = S_x[time_x] - S_y[time_y]
        chosen = (np.arange(len(signal))+1) % period == 0
        all_signal = np.concatenate((all_signal, signal[chosen]), axis=0)
    save(all_signal, HEAD_PATH+"/all_signals/"+product_x+"_"+product_y+"."+signal_name+".pkl")


In [ ]:
%%time
result = par_get_arb_all_signal("trade.imb.4096",  file_list=all_dates, product_x="rb", 
                   product_y = "hc", period=4096, HEAD_PATH=HEAD_PATH, SAVE_PATH=SAVE_PATH)

In [ ]:
from collections import OrderedDict
def get_arb_signal_pnl(file, product_x, product_y, signal_name, thre_mat, reverse=1, tranct=1.1e-4, max_spread_x=1.1,
                       max_spread_y=1.1,
                       tranct_ratio=True, HEAD_PATH=HEAD_PATH, SAVE_PATH = SAVE_PATH, rebate=0):
    ## load data
    data_x = load(HEAD_PATH+"\\pkl_tick\\"+product_x+"\\"+file)
    data_y = load(HEAD_PATH+"\\pkl_tick\\"+product_y+"\\"+file)
    [time_x, time_y] = load(HEAD_PATH+"\\comb_time\\"+product_x+"_"+product_y+"\\"+file)
    S_x = load(SAVE_PATH+"\\factor_pkl\\"+product_x+"\\"+signal_name+"\\"+file)
    S_y = load(SAVE_PATH+"\\factor_pkl\\"+product_y+"\\"+signal_name+"\\"+file)
    S = S_x[time_x] - S_y[time_y]                  
    pred = S*reverse
    data_y = data_y[time_y].reset_index(drop=True)
    data_x = data_x[time_x].reset_index(drop=True)
    good_x = data_x["good"]
    good_y = data_y["good"]
    pred = pred[good_x & good_y]
    data_x = data_x[good_x & good_y].reset_index(drop=True)    
    data_y = data_y[good_x & good_y].reset_index(drop=True)
    result = pd.DataFrame(data=OrderedDict([("open", thre_mat["open"].values), ("close", thre_mat["close"].values),
                               ("num", 0), ("avg.pnl", 0), ("pnl", 0), ("avg.ret", 0), ("ret", 0)]), 
                          index=thre_mat.index)
    count = 0
    cur_spread_x = data_x["ask"]-data_x["bid"]
    cur_spread_y = data_y["ask"]-data_y["bid"]
    for thre in thre_mat.iterrows():
        count = count+1
        buy = pred>thre[1]["open"]
        sell = pred<-thre[1]["open"]
        signal = pd.Series(data=0, index=data_x.index)
        position = signal.copy()
        signal[buy] = 1
        signal[sell] = -1
        scratch = -thre[1]["close"]
        position_pos = pd.Series(data=np.nan, index=data_x.index)
        position_pos.iloc[0] = 0
        position_pos[(signal==1) & (data_x["next.ask"]>0) & (data_x["next.bid"]>0) & (cur_spread_x<max_spread_x) &
                    (data_y["next.ask"]>0) & (data_y["next.bid"]>0) & (cur_spread_y<max_spread_y)] = 1
        position_pos[(pred< -scratch) & (data_x["next.bid"]>0) & (data_y["next.ask"]>0) & (cur_spread_x<max_spread_x) &
                    (cur_spread_y<max_spread_y)] = 0
        position_pos.ffill(inplace=True)
        pre_pos = position_pos.shift(1)
        notional_position_pos_x = pd.Series(data=0, index=data_x.index)
        notional_position_pos_y = pd.Series(data=0, index=data_y.index)
        notional_position_pos_x[position_pos==1] = 1
        notional_position_pos_x[(position_pos==1) & (pre_pos==1)] = np.nan
        notional_position_pos_y[position_pos==1] = 1
        notional_position_pos_y[(position_pos==1) & (pre_pos==1)] = np.nan
        notional_position_pos_x[(notional_position_pos_x==1)] = 1/data_x["next.ask"][(notional_position_pos_x==1)]
        notional_position_pos_y[(notional_position_pos_y==1)] = -1/data_y["next.bid"][(notional_position_pos_y==1)]
        notional_position_pos_x.ffill(inplace=True)
        notional_position_pos_y.ffill(inplace=True)
        position_neg = pd.Series(data=np.nan, index=data_x.index)
        position_neg.iloc[0] = 0
        position_neg[(signal==-1) & (data_x["next.ask"]>0) & (data_x["next.bid"]>0) & (cur_spread_x<max_spread_x) &
                    (data_y["next.ask"]>0) & (data_y["next.bid"]>0) & (cur_spread_y<max_spread_y)] = -1
        position_neg[(pred> scratch) & (data_x["next.ask"]>0) & (data_y["next.bid"]>0) & (cur_spread_y<max_spread_y)] = 0
        position_neg.ffill(inplace=True)
        pre_neg = position_neg.shift(1)
        notional_position_neg_x = pd.Series(data=0, index=data_x.index)
        notional_position_neg_y = pd.Series(data=0, index=data_y.index)
        notional_position_neg_x[position_neg==-1] = -1
        notional_position_neg_x[(position_neg==-1) & (pre_neg==-1)] = np.nan
        notional_position_neg_y[position_neg==-1] = -1
        notional_position_neg_y[(position_neg==-1) & (pre_neg==-1)] = np.nan
        notional_position_neg_x[(notional_position_neg_x==-1)] = -1/data_x["next.bid"][(notional_position_neg_x==-1)]
        notional_position_neg_y[(notional_position_neg_y==-1)] = 1/data_y["next.ask"][(notional_position_neg_y==-1)]
        notional_position_neg_x.ffill(inplace=True)
        notional_position_neg_y.ffill(inplace=True)
        position = position_pos + position_neg
        notional_position_x = notional_position_pos_x+notional_position_neg_x
        notional_position_y = notional_position_pos_y+notional_position_neg_y
        #position[n_bar-1] = 0
        position.iloc[0] = 0
        position.iloc[-10:] = 0
        notional_position_x.iloc[0] = 0
        notional_position_y.iloc[0] = 0
        notional_position_x.iloc[-10:] = 0
        notional_position_y.iloc[-10:] = 0
        notional_change_pos_x = notional_position_x-notional_position_x.shift(1)
        notional_change_pos_y = notional_position_y-notional_position_y.shift(1)
        notional_change_pos_x.iloc[0] = 0
        notional_change_pos_y.iloc[0] = 0
        change_pos = position - position.shift(1)
        change_pos.iloc[0] = 0
        change_base_x = pd.Series(data=0, index=data_x.index)
        change_base_y = pd.Series(data=0, index=data_y.index)

        change_buy = change_pos>0
        change_sell = change_pos<0        
        if (tranct_ratio):
            change_base_x[change_buy] = data_x["next.ask"][change_buy]*(1+tranct)
            change_base_x[change_sell] = data_x["next.bid"][change_sell]*(1-tranct)
            change_base_y[change_buy] = data_y["next.bid"][change_buy]*(1-tranct)
            change_base_y[change_sell] = data_y["next.ask"][change_sell]*(1+tranct)
        else:
            change_base_x[change_buy] = data_x["next.ask"][change_buy]+tranct
            change_base_x[change_sell] = data_x["next.bid"][change_sell]-tranct
            change_base_y[change_buy] = data_y["next.bid"][change_buy]-tranct
            change_base_y[change_sell] = data_y["next.ask"][change_sell]+tranct

        final_pnl = -sum(change_base_x*change_pos)+sum(change_base_y*change_pos)
        ret = -sum(change_base_x*notional_change_pos_x)-sum(change_base_y*notional_change_pos_y)
        num = sum((position!=0) & (change_pos!=0))
        if num == 0:
            result.loc[thre[0], ("num", "avg.pnl", "pnl", "avg.ret", "ret")] = (0,0,0,0,0)
            return result
        else:
            avg_pnl = np.divide(final_pnl, num)
            avg_ret = np.divide(ret,num)
            result.loc[thre[0], ("num", "avg.pnl", "pnl", "avg.ret", "ret")] = (num, avg_pnl, final_pnl, avg_ret,ret)
    return result


**CTA**

In [ ]:
import time
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier)
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [ ]:
period = 4096

# train and test
train_sample = np.array(date_list) < "201901"
test_sample = np.array(date_list) > "201901"

# 记录时间的字符串和时间格式
date_str = date_list
format_dates = np.array([pd.to_datetime(d) for d in date_str])
